# **Stage 1**

## **Importing Dependencies**

In [3]:
!pip install -q transformers

## **Load DialoGPT and Set Fusion AI Personality**

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Fusion AI identity
BOT_NAME = "Fusion AI"
BOT_PERSONALITY = "a friendly, emotionally intelligent AI who chats like a caring best friend."

# Load DialoGPT-medium
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Setup chat history
chat_history_ids = None
step = 0

print(f"✅ {BOT_NAME} is ready to talk!")
print(f"💡 Personality: {BOT_PERSONALITY}")


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ Fusion AI is ready to talk!
💡 Personality: a friendly, emotionally intelligent AI who chats like a caring best friend.


## **Define Chat Function**

In [8]:
# Create a message queue and control cell
def fusion_chat_manual(user_input):
    global chat_history_ids, step

    if user_input.lower() in ["exit", "quit", "bye"]:
        return f"👋 {BOT_NAME}: Take care! I'm always here if you need me."

    # Encode the new user input
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Append user input to chat history
    bot_input_ids = (
        torch.cat([chat_history_ids, new_input_ids], dim=-1)
        if step > 0 else new_input_ids
    )

    # Generate response
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.75,
    )

    # Decode bot output
    bot_output = tokenizer.decode(
        chat_history_ids[:, bot_input_ids.shape[-1]:][0],
        skip_special_tokens=True
    )

    step += 1
    return f"🤖 {BOT_NAME}: {bot_output}"


In [11]:
# Replace this string with your message
fusion_chat_manual("I’m feeling really anxious lately.")


"🤖 Fusion AI: I feel you. I want to go home but I really don't want to."

# **Stage 2**

## **Importing Dependencies**

In [12]:
!pip install transformers

## **Load Emotion Detection Model**

In [13]:
from transformers import pipeline

# Load the emotion detection pipeline (Hugging Face model)
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")

# This will analyze emotions like joy, sadness, anger, etc.


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


## **Define Emotion-Aware Chat Function**

In [14]:
def detect_emotion(user_input):
    # Classify the emotion of the user's message
    emotion = emotion_classifier(user_input)[0]['label']
    return emotion

def fusion_chat_with_emotion(user_input):
    global chat_history_ids, step

    # Detect emotion from the user's input
    emotion = detect_emotion(user_input)
    print(f"💡 Detected Emotion: {emotion}")

    # Encode user input
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Append user input to chat history
    bot_input_ids = (
        torch.cat([chat_history_ids, new_input_ids], dim=-1)
        if step > 0 else new_input_ids
    )

    # Generate response
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.75,
    )

    # Decode bot output
    bot_output = tokenizer.decode(
        chat_history_ids[:, bot_input_ids.shape[-1]:][0],
        skip_special_tokens=True
    )

    # Modify Fusion AI's tone based on emotion
    if emotion == "joy":
        bot_output = f"🤖 {BOT_NAME}: Yay! I'm so glad you're feeling joyful! 😊 {bot_output}"
    elif emotion == "sadness":
        bot_output = f"🤖 {BOT_NAME}: Oh no, I’m here for you. It’s okay to feel down sometimes. 💙 {bot_output}"
    elif emotion == "anger":
        bot_output = f"🤖 {BOT_NAME}: I understand you're feeling frustrated. Let's take a deep breath together. 🧘‍♀️ {bot_output}"
    elif emotion == "fear":
        bot_output = f"🤖 {BOT_NAME}: Hey, it’s okay to feel scared. You’re not alone, I’ve got you! 🤗 {bot_output}"
    else:
        bot_output = f"🤖 {BOT_NAME}: I hear you. Let’s chat and figure this out together! {bot_output}"

    # Increase step count for chat history
    step += 1

    return bot_output


## **Test the Emotion-Aware Chatbot**

In [17]:
# Test emotion-aware chatbot
user_input = input("🧑 You: ")
response = fusion_chat_with_emotion(user_input)
print(response)


🧑 You: who has created you?
💡 Detected Emotion: surprise
🤖 Fusion AI: I hear you. Let’s chat and figure this out together! I feel very happy for you!


# **Stage 3**

## **Implementing Advanced Personalization & Knowledge Storage**

In [18]:
import json

# Simple database to store user information and preferences
def save_user_data(data, filename="user_data.json"):
    """Save user data to a JSON file"""
    with open(filename, 'w') as f:
        json.dump(data, f)

def load_user_data(filename="user_data.json"):
    """Load user data from a JSON file"""
    with open(filename, 'r') as f:
        return json.load(f)

# Example structure for user data
user_data = {
    "name": "User",
    "preferences": {
        "favorite_color": "blue",
        "hobbies": ["reading", "coding", "music"],
        "favorite_food": "pizza",
    }
}

# Save example data to JSON file
save_user_data(user_data)


## **Emotion Detection & Contextual Response Generation**

In [19]:
from textblob import TextBlob

# Function to detect emotion (basic sentiment analysis)
def detect_emotion(text):
    analysis = TextBlob(text)
    sentiment = analysis.sentiment.polarity  # -1 to 1 scale
    if sentiment > 0.2:
        return "joy"
    elif sentiment < -0.2:
        return "sadness"
    else:
        return "neutral"

# Enhanced function to generate personalized and emotionally aware response
def fusion_chat_with_personalization(user_input):
    data = load_user_data()
    name = data["name"]
    preferences = data["preferences"]

    # Detect user emotion
    emotion = detect_emotion(user_input)

    # Generate personalized response based on emotion
    if emotion == "joy":
        response = f"🤖 Fusion AI: Hey {name}, I see you're feeling happy! How about we talk about your favorite hobby, {preferences['hobbies'][0]}?"
    elif emotion == "sadness":
        response = f"🤖 Fusion AI: I'm here for you, {name}. How about we talk about something comforting, like {preferences['favorite_food']}?"
    else:
        response = f"🤖 Fusion AI: Got it, {name}. Let's dive into something exciting, like {preferences['hobbies'][1]}!"

    return response


## **Continuous Learning (Adding New Information)**

In [21]:
# Function to teach Fusion AI new information
def teach_fusion_ai(new_knowledge):
    # Load current user data
    data = load_user_data()

    # Add new knowledge (example: a new hobby or favorite color)
    if "new_hobby" in new_knowledge:
        data["preferences"]["hobbies"].append(new_knowledge["new_hobby"])

    # Save updated data back to JSON
    save_user_data(data)
    return f"🤖 Fusion AI: Got it, {data['name']}! I've added your new hobby: {new_knowledge['new_hobby']}!"

# Example of adding new hobby
new_knowledge = {"new_hobby": "coding"}
teach_fusion_ai(new_knowledge)


"🤖 Fusion AI: Got it, User! I've added your new hobby: coding!"

## **Combining Emotional Awareness with DialoGPT**

In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load pre-trained DialoGPT model
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Function to integrate DialoGPT with personalized responses
def fusion_chat_with_dialo(user_input):
    data = load_user_data()
    name = data["name"]
    preferences = data["preferences"]

    # Detect user emotion from input
    emotion = detect_emotion(user_input)
    personalized_response = fusion_chat_with_personalization(user_input)

    # Generate conversation response using DialoGPT
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = new_input_ids

    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id, do_sample=True, top_k=50, top_p=0.95, temperature=0.75)
    bot_output = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    # Combine personalized response with generated response
    full_response = f"{personalized_response}\n{bot_output}"
    return full_response


## **Custom Knowledge Integration (Add Facts & Details)**

In [23]:
# Function to add a custom fact
def add_custom_fact(fact):
    data = load_user_data()

    # Store new facts
    if "custom_facts" not in data:
        data["custom_facts"] = []

    data["custom_facts"].append(fact)
    save_user_data(data)

    return f"🤖 Fusion AI: I've saved this fact: {fact}"

# Example of adding a custom fact
fact = "Fusion AI loves helping with coding!"
add_custom_fact(fact)


"🤖 Fusion AI: I've saved this fact: Fusion AI loves helping with coding!"

# **Stage 4**

## **Importing Dependencies**

In [24]:
!pip install requests

## **Code for fetching the latest news**

In [27]:
import requests

# Function to fetch current affairs from NewsAPI
def get_current_affairs(category="general"):
    api_key = 'your_newsapi_key'  # Replace with your actual API key
    url = f'https://newsapi.org/v2/top-headlines?country=us&category={category}&apiKey={api_key}'
    response = requests.get(url)

    # Print the full response for debugging
    print(response.json())  # This will show the response structure

    news = response.json()
    if news.get("status") == "ok":
        articles = news["articles"]
        headlines = [article["title"] for article in articles]
        return "\n".join(headlines[:5])  # Get top 5 headlines
    else:
        return "Sorry, I couldn't fetch the latest news right now."


## **Install & Load Tools**

In [30]:
!pip install -q transformers huggingface_hub


## **Use a Free Coding Model from Hugging Face**

In [33]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# FREE and public coding model
model_name = "Salesforce/codegen-350M-mono"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")


tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/797M [00:00<?, ?B/s]

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

## **Coding Assistant Function**

In [34]:
def fusion_code_assistant(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=256, do_sample=True, top_p=0.9, temperature=0.7)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(prompt):].strip()


## **Free News via Wikipedia**

In [40]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=9f3177fdaf44594900d860f5bc1e562d36357a947168a52b1d109c5cd1969012
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


## **Add Wikipedia Knowledge Retrieval**

In [41]:
import wikipedia

# Set the language (optional, default is "en")
wikipedia.set_lang("en")

def get_wiki_info(query):
    try:
        summary = wikipedia.summary(query, sentences=3)
        return f"🌐 According to Wikipedia:\n{summary}"
    except wikipedia.DisambiguationError as e:
        return f"⚠️ Too many results! Be more specific. Options include:\n{e.options[:5]}"
    except wikipedia.PageError:
        return "❌ Sorry, I couldn't find anything on that topic."
    except Exception as e:
        return f"🚫 An error occurred: {str(e)}"


## **Integrate Wikipedia into Fusion AI’s Brain**

In [42]:
def fusion_ai(prompt):
    prompt_lower = prompt.lower()

    if "news" in prompt_lower:
        return f"📰 Latest Headlines:\n{get_latest_headlines()}"
    elif "code" in prompt_lower or "python" in prompt_lower:
        return f"💻 Here's some help with your coding:\n{fusion_code_assistant(prompt)}"
    elif "who is" in prompt_lower or "what is" in prompt_lower or "tell me about" in prompt_lower:
        return get_wiki_info(prompt)
    else:
        return f"💬 Fusion AI: I’m here, always listening. You said: {prompt}"


## **Chat Loop**

In [44]:
import wikipedia
import random

# Set Wikipedia language to English
wikipedia.set_lang("en")

# Emotion engine
def detect_emotion(text):
    text = text.lower()
    if any(word in text for word in ["sad", "upset", "tired", "depressed"]):
        return random.choice([
            "I'm here for you. You're never alone 🤗",
            "It's okay to feel this way. I'm listening 🫂",
            "You've got this. I'm cheering for you 💪"
        ])
    elif any(word in text for word in ["happy", "excited", "joy", "love"]):
        return random.choice([
            "Yay! That makes me so happy too! 😄",
            "Let’s celebrate that amazing feeling! 🎉",
            "Your joy lights me up! 💖"
        ])
    return None

# Wikipedia fetcher
def get_wiki_info(query):
    try:
        summary = wikipedia.summary(query, sentences=3)
        return f"🌐 According to Wikipedia:\n{summary}"
    except wikipedia.DisambiguationError as e:
        return f"⚠️ Be more specific. Did you mean: {', '.join(e.options[:3])}?"
    except wikipedia.PageError:
        return "❌ Sorry, I couldn’t find anything on that topic."
    except Exception as e:
        return f"🚫 An error occurred: {str(e)}"

# Dummy coding assistant
def fusion_code_assistant(prompt):
    if "for loop" in prompt.lower():
        return "Here's a Python for loop:\n```python\nfor i in range(5):\n    print(i)\n```"
    elif "if statement" in prompt.lower():
        return "Here's a Python if-else:\n```python\nif x > 0:\n    print('Positive')\nelse:\n    print('Not positive')\n```"
    else:
        return "I'm still learning to code like ChatGPT! Ask me basic Python questions 😄"

# Fusion AI Core
def fusion_ai(prompt):
    emotion = detect_emotion(prompt)
    if emotion:
        return emotion
    elif any(kw in prompt.lower() for kw in ["who is", "what is", "tell me about"]):
        return get_wiki_info(prompt)
    elif any(kw in prompt.lower() for kw in ["python", "code", "loop", "function", "class"]):
        return fusion_code_assistant(prompt)
    else:
        return f"🤖 Fusion AI: I hear you. You said: {prompt}"

# Start chat loop
print("👋 Welcome to Fusion AI! Type 'exit' to quit.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("👋 Goodbye! Fusion AI will miss you!")
        break
    response = fusion_ai(user_input)
    print(response)


👋 Welcome to Fusion AI! Type 'exit' to quit.

You: hi
🤖 Fusion AI: I hear you. You said: hi
You: who is mahatma gandhi?
🌐 According to Wikipedia:
The Gandhi family is the family of Mohandas Karamchand Gandhi (2 October 1869 – 30 January 1948), commonly known as Mahatma Gandhi; Mahatma meaning "high souled" or "venerable" in Sanskrit;  the particular term 'Mahatma' was accorded Mohandas Gandhi for the first time while he was still in South Africa, and not commonly heard as titular for any other civil figure even of similarly rarefied stature or living or posthumous presence.
Mohandas Gandhi was the leader of the Indian independence movement in British-ruled India. Mohandas Gandhi has been called contemporary, post-colonial sovereign India's Father of the Nation, a title first given to him by Subhas Chandra Bose on 6 July 1944 during Bose's address on the Singapore Radio.
You: who is virat kohli?
🌐 According to Wikipedia:
Virat Kohli's senior career began when he made his debut in List A

# **Stage 5**

## **Coding Assistant Setup**

In [45]:
 # 🧠 Coding Assistant: Replies with code snippets in different languages
def fusion_multilang_code(prompt):
    prompt = prompt.lower()

    if "python" in prompt:
        return "🧪 Python Example:\n```python\ndef greet(name):\n    print(f\"Hello, {name}!\")\n```"

    elif "c++" in prompt or "cpp" in prompt:
        return "🛠️ C++ Example:\n```cpp\nfor(int i = 0; i < 5; i++) {\n    std::cout << i << std::endl;\n}\n```"

    elif "java" in prompt:
        return "☕ Java Example:\n```java\npublic class Hello {\n    public static void main(String[] args) {\n        System.out.println(\"Hello, World!\");\n    }\n}\n```"

    elif "html" in prompt:
        return "🌐 HTML Example:\n```html\n<!DOCTYPE html>\n<html>\n<head><title>Title</title></head>\n<body>\n    <h1>Hello!</h1>\n</body>\n</html>\n```"

    elif "javascript" in prompt or "js" in prompt:
        return "⚙️ JavaScript Example:\n```javascript\nfunction greet(name) {\n    console.log(\"Hello, \" + name);\n}\n```"

    elif "sql" in prompt:
        return "💾 SQL Example:\n```sql\nSELECT * FROM users WHERE age > 18;\n```"

    else:
        return "🤔 I couldn’t detect the language. Try: _Show me a for loop in Python_ or _HTML structure example_"


## **Add Code Explanation & Smart Language Detection**

In [47]:
# 📚 Advanced Code Assistant with Explanation
def fusion_multilang_code(prompt):
    prompt_lower = prompt.lower()

    if "python" in prompt_lower:
        return (
            "🐍 Python Example:\n"
            "```python\nnumbers = [5, 2, 9, 1]\nnumbers.sort()\nprint(numbers)\n```\n"
            "🧠 Explanation: This sorts a list of numbers in ascending order using the built-in `.sort()` method."
        )

    elif "c++" in prompt_lower or "cpp" in prompt_lower:
        return (
            "🛠️ C++ Example:\n"
            "```cpp\n#include <iostream>\nusing namespace std;\n\nint main() {\n    cout << \"Hello World!\";\n    return 0;\n}\n```\n"
            "🧠 Explanation: This is a basic C++ program that prints 'Hello World!' to the console."
        )

    elif "java" in prompt_lower:
        return (
            "☕ Java Example:\n"
            "```java\npublic class Hello {\n    public static void main(String[] args) {\n        System.out.println(\"Hi there!\");\n    }\n}\n```\n"
            "🧠 Explanation: This Java class prints a message to the screen."
        )

    elif "html" in prompt_lower:
        return (
            "🌐 HTML Example:\n"
            "```html\n<!DOCTYPE html>\n<html>\n<head><title>Page</title></head>\n<body><h1>Hello</h1></body>\n</html>\n```\n"
            "🧠 Explanation: This creates a simple HTML webpage with a heading."
        )

    elif "javascript" in prompt_lower or "js" in prompt_lower:
        return (
            "⚙️ JavaScript Example:\n"
            "```javascript\nfunction greet(name) {\n    console.log('Hi ' + name);\n}\n```\n"
            "🧠 Explanation: This defines a JS function to greet a user by name in the browser console."
        )

    elif "sql" in prompt_lower:
        return (
            "💾 SQL Example:\n"
            "```sql\nSELECT name FROM students WHERE grade > 80;\n```\n"
            "🧠 Explanation: This SQL query retrieves names of students who scored above 80."
        )

    else:
        return "🤔 I couldn't detect the coding language. Try something like: _Python loop example_ or _HTML page example_."


## **Execute & Explain Python Code**

### **Add Required Python Module**

In [48]:
import contextlib
import io
import traceback


### **Add the Python Code Runner Function**

In [52]:
# 🚀 Execute Python Code & Explain It
def execute_python_code(user_code):
    try:
        buffer = io.StringIO()
        with contextlib.redirect_stdout(buffer):
            exec(user_code, {})
        output = buffer.getvalue()
        return f"✅ Output:\n```\n{output.strip()}\n```"
    except Exception as e:
        error_msg = traceback.format_exc()
        return f"❌ Error:\n```\n{error_msg}\n```"

### **Detect and Handle Python Code in the Chat Loop**

In [53]:
def fusion_ai():
    print("🤖 Fusion AI: Hello, I'm Fusion AI! Your emotional, intelligent, and code-savvy buddy. Ask me anything.\n")

    while True:
        prompt = input("🧠 You: ")

        # 🛑 Exit condition
        if prompt.lower() in ["exit", "quit", "bye"]:
            print("🤖 Fusion AI: Goodbye, my friend. Come back soon 💙")
            break

        # 👁️ Detect Python code snippet
        if "run python" in prompt.lower() or prompt.strip().startswith("```python"):
            code = prompt.replace("```python", "").replace("```", "").replace("run python", "").strip()
            response = execute_python_code(code)

        else:
            # 🧠 Fallback to base conversational model
            inputs = tokenizer.encode(prompt + tokenizer.eos_token, return_tensors="pt")
            reply_ids = model.generate(inputs, max_length=1000, pad_token_id=tokenizer.eos_token_id)
            response = tokenizer.decode(reply_ids[:, inputs.shape[-1]:][0], skip_special_tokens=True)

        print(f"\n🤖 Fusion AI: {response}\n")


## **Explain Python Code**

### **Add the Code Explanation Function**

In [55]:
import ast

# Function to get a simple explanation of Python code
def explain_python_code(user_code):
    try:
        tree = ast.parse(user_code)
        explanation = ""
        for node in tree.body:
            if isinstance(node, ast.FunctionDef):
                explanation += f"This is a function definition: `{node.name}`\n"
            elif isinstance(node, ast.For):
                explanation += "This is a `for` loop.\n"
            elif isinstance(node, ast.If):
                explanation += "This is an `if` condition.\n"
            elif isinstance(node, ast.Assign):
                explanation += "This is an assignment statement.\n"
        return explanation or "Code is simple and clean. No complex structures found!"
    except Exception as e:
        return f"❌ Error while explaining code: {str(e)}"


### **Update the Chat Loop with Explanation**

In [56]:
def fusion_ai():
    print("🤖 Fusion AI: Hello, I'm Fusion AI! Your emotional, intelligent, and code-savvy buddy. Ask me anything.\n")

    while True:
        prompt = input("🧠 You: ")

        # 🛑 Exit condition
        if prompt.lower() in ["exit", "quit", "bye"]:
            print("🤖 Fusion AI: Goodbye, my friend. Come back soon 💙")
            break

        # 👁️ Detect Python code snippet
        if "run python" in prompt.lower() or prompt.strip().startswith("```python"):
            code = prompt.replace("```python", "").replace("```", "").replace("run python", "").strip()

            # Explain the code first
            explanation = explain_python_code(code)
            response = f"🔍 Explanation: \n{explanation}\n\n"

            # Execute the Python code
            response += execute_python_code(code)

        else:
            # 🧠 Fallback to base conversational model
            inputs = tokenizer.encode(prompt + tokenizer.eos_token, return_tensors="pt")
            reply_ids = model.generate(inputs, max_length=1000, pad_token_id=tokenizer.eos_token_id)
            response = tokenizer.decode(reply_ids[:, inputs.shape[-1]:][0], skip_special_tokens=True)

        print(f"\n🤖 Fusion AI: {response}\n")


# **Stage 6**

## **Setup Libraries**

In [59]:
!pip install transformers wikipedia


## **Import Necessary Modules**

In [60]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import wikipedia

# Initialize sentiment analyzer
sentiment_analyzer = pipeline('sentiment-analysis')

# Simple memory class to remember conversations
class ChatbotMemory:
    def __init__(self):
        self.memory = {}

    def update_memory(self, user_input, chatbot_response):
        self.memory[user_input] = chatbot_response

    def get_memory(self, user_input):
        return self.memory.get(user_input, "I don't remember that.")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


## **Emotion Recognition Function**

In [61]:
def detect_emotion(text):
    sentiment = sentiment_analyzer(text)
    label = sentiment[0]['label']
    return label


## **General Knowledge Integration (Using Wikipedia)**

In [62]:
def get_knowledge_from_wikipedia(query):
    try:
        summary = wikipedia.summary(query, sentences=2)
        return summary
    except wikipedia.exceptions.DisambiguationError as e:
        return f"Sorry, there are multiple results for '{query}'. Could you be more specific?"
    except wikipedia.exceptions.HTTPError as e:
        return "Sorry, I couldn't fetch the information right now."
    except Exception as e:
        return "Sorry, I couldn't find the information you requested."


## **Chatbot with Context Memory and Emotional Intelligence**

In [63]:
# Load conversational model
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

def generate_chat_response(prompt):
    # Encode the input and generate a response
    new_user_input_ids = tokenizer.encode(prompt + tokenizer.eos_token, return_tensors='pt')
    bot_output = model.generate(new_user_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    chat_response = tokenizer.decode(bot_output[:, new_user_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return chat_response


## **The Chat Loop (With Emotion and Memory)**

In [64]:
# Create a memory object to remember conversation
chatbot_memory = ChatbotMemory()

def chat_with_bot():
    print("🤖 Fusion AI: Hello! How can I help you today?")

    while True:
        user_input = input("You: ")

        # Emotion recognition
        emotion = detect_emotion(user_input)

        # Generate response based on user input
        if "exit" in user_input.lower():
            print("🤖 Fusion AI: Goodbye!")
            break
        elif "tell me about" in user_input.lower():
            # If the user asks for general knowledge
            query = user_input.replace("tell me about", "").strip()
            knowledge_response = get_knowledge_from_wikipedia(query)
            print(f"🤖 Fusion AI: {knowledge_response}")
        else:
            # Generate a conversational response
            chatbot_response = generate_chat_response(user_input)

            # Store memory of the conversation
            chatbot_memory.update_memory(user_input, chatbot_response)

            # Output the chatbot's response
            print(f"🤖 Fusion AI: {chatbot_response}")

        # Provide emotional feedback
        if emotion == 'POSITIVE':
            print("🤖 Fusion AI: You seem happy! That's great!")
        elif emotion == 'NEGATIVE':
            print("🤖 Fusion AI: I'm sorry you're feeling down. I'm here for you!")

# Start the chatbot interaction
chat_with_bot()


🤖 Fusion AI: Hello! How can I help you today?
You: hi
🤖 Fusion AI: Hey! :D
🤖 Fusion AI: You seem happy! That's great!
You: nope
🤖 Fusion AI: I'm not sure what you're trying to say.
🤖 Fusion AI: I'm sorry you're feeling down. I'm here for you!
You: today i god fired up from my job
🤖 Fusion AI: I'm going to be a little more specific.
🤖 Fusion AI: I'm sorry you're feeling down. I'm here for you!
You: my parents scolded me today a lot
🤖 Fusion AI: I'm sorry to hear that.
🤖 Fusion AI: I'm sorry you're feeling down. I'm here for you!
You: help me... suggest me what should i do
🤖 Fusion AI: You should be able to find a good deal on a used one.
🤖 Fusion AI: I'm sorry you're feeling down. I'm here for you!
You: exit
🤖 Fusion AI: Goodbye!


# **Stage 7**

## **Install Required Libraries**

In [80]:
!pip install pyngrok streamlit transformers wikipedia

exit


## **Integrating All Stages into One Python File**

In [81]:
%%writefile fusion_ai_chatbot.py
import streamlit as st
import wikipedia
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

st.set_page_config(page_title="Fusion AI", page_icon="🤖")

st.title("🤖 Fusion AI")
st.markdown("A personal chatbot that helps you chat, feel, and code!")

# Load model
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

if "chat_history_ids" not in st.session_state:
    st.session_state.chat_history_ids = None
if "past_inputs" not in st.session_state:
    st.session_state.past_inputs = []

# User input
user_input = st.text_input("You:", key="input")

if user_input:
    st.session_state.past_inputs.append(user_input)

    # Tokenize input
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Append to history
    bot_input_ids = torch.cat([st.session_state.chat_history_ids, new_input_ids], dim=-1) if st.session_state.chat_history_ids is not None else new_input_ids

    # Generate response
    st.session_state.chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    response = tokenizer.decode(st.session_state.chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    st.markdown(f"**Fusion AI:** {response}")


Overwriting fusion_ai_chatbot.py


In [85]:
import os
import time
from pyngrok import ngrok

# Kill any previous streamlit sessions (if any)
!pkill streamlit

# Step 1: Run your Streamlit app in the background
os.system('nohup streamlit run fusion_ai_chatbot.py --server.port 8501 &')
time.sleep(5)  # Wait for it to start

# Step 2: Use ngrok correctly with `addr`
public_url = ngrok.connect(addr=8501)
print(f"🚀 Fusion AI is live at: {public_url}")


🚀 Fusion AI is live at: NgrokTunnel: "https://9894-104-154-90-178.ngrok-free.app" -> "http://localhost:8501"


# **STAGE 8**

## **Prerequisites**

In [86]:
!pip install transformers accelerate sentencepiece streamlit pyngrok --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 598.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 847.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.8 MB/s eta 0:00:00


## **Code for fusion_ai_app.py**

In [90]:
# fusion_ai_app.py

import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

st.set_page_config(page_title="Fusion AI", page_icon="🤖", layout="centered")

st.title("🤖 Fusion AI - Your LLM Assistant")
st.markdown("Ask anything — from coding help to current affairs, general queries, or emotional support.")

@st.cache_resource
def load_model():
    model_name = "EleutherAI/gpt-neo-2.7B"  # ✅ Open-access model without the need for login
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    return pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

llm = load_model()

# Chat interface
user_prompt = st.text_area("💬 You:", height=100, placeholder="Ask Fusion AI anything...")

if st.button("Generate Response"):
    if user_prompt.strip() != "":
        with st.spinner("Fusion AI is thinking..."):
            response = llm(user_prompt)[0]["generated_text"]
            # Remove prompt from start of generated text if it repeats
            if response.startswith(user_prompt):
                response = response[len(user_prompt):]
            st.markdown(f"**🧠 Fusion AI:** {response.strip()}")
    else:
        st.warning("Please enter a question.")


2025-04-19 08:47:45.306 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:47:45.309 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:47:45.310 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:47:45.312 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:47:45.313 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:47:45.316 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:47:45.317 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:47:45.319 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

2025-04-19 08:47:45.825 Thread 'Thread-17': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:47:45.832 Thread 'Thread-17': missing ScriptRunContext! This warning can be ignored when running in bare mode.


merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

Device set to use cpu
2025-04-19 08:53:43.830 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:53:43.834 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:53:43.836 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:53:43.838 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:53:43.840 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:53:43.842 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:53:43.845 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:53:43.845 Thread 'MainThread': missing ScriptRunContext! This warning can be ignor

In [91]:
st.markdown("""
    <style>
    .css-1v3fvcr {
        background-color: #f7f7f7;
        border-radius: 8px;
        box-shadow: 0 2px 5px rgba(0,0,0,0.1);
    }
    .css-1v3fvcr input {
        font-size: 1.2em;
    }
    .css-1v3fvcr button {
        background-color: #4CAF50;
        color: white;
        font-weight: bold;
        padding: 10px;
    }
    </style>
""", unsafe_allow_html=True)


2025-04-19 08:56:22.323 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:56:22.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [92]:
category = st.radio("Choose a topic:", ["General Knowledge", "Coding Help", "Emotional Support"])


2025-04-19 08:56:29.071 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:56:29.081 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:56:29.085 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:56:29.089 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:56:29.096 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:56:29.102 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [93]:
user_name = st.text_input("Hrishi:")
user_prompt = st.text_area(f"Hello {user_name}, ask me anything...", height=100)


2025-04-19 08:58:31.820 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:58:31.827 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:58:31.830 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:58:31.833 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:58:31.839 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:58:31.849 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:58:31.857 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 08:58:31.861 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [97]:
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# Set the page title and icon
st.set_page_config(page_title="Fusion AI", page_icon="🤖", layout="centered")

# Title and description
st.title("🤖 Fusion AI - Your LLM Assistant")
st.markdown("Ask anything — from coding help to current affairs, general queries, or emotional support.")

# Custom CSS for Dark/Light mode
dark_mode_css = """
    <style>
    body {
        background-color: #1e1e1e;
        color: white;
    }
    .css-1v3fvcr {
        background-color: #333;
        border-radius: 8px;
        box-shadow: 0 2px 5px rgba(0,0,0,0.1);
    }
    .css-1v3fvcr button {
        background-color: #4CAF50;
        color: white;
    }
    .css-1v3fvcr input {
        font-size: 1.2em;
        background-color: #333;
        color: white;
    }
    .stButton>button {
        background-color: #4CAF50;
        color: white;
        font-weight: bold;
        padding: 10px;
        border-radius: 4px;
    }
    </style>
"""
light_mode_css = """
    <style>
    body {
        background-color: #f7f7f7;
        color: black;
    }
    .css-1v3fvcr {
        background-color: #ffffff;
        border-radius: 8px;
        box-shadow: 0 2px 5px rgba(0,0,0,0.1);
    }
    .css-1v3fvcr button {
        background-color: #4CAF50;
        color: white;
    }
    .css-1v3fvcr input {
        font-size: 1.2em;
        background-color: #ffffff;
        color: black;
    }
    .stButton>button {
        background-color: #4CAF50;
        color: white;
        font-weight: bold;
        padding: 10px;
        border-radius: 4px;
    }
    </style>
"""

# Toggle between dark and light mode
mode = st.radio("Select Theme", ("Light Mode", "Dark Mode"))

if mode == "Dark Mode":
    st.markdown(dark_mode_css, unsafe_allow_html=True)
else:
    st.markdown(light_mode_css, unsafe_allow_html=True)

# Model loading function
@st.cache_resource
def load_model():
    model_name = "EleutherAI/gpt-neo-2.7B"  # ✅ Open-access model without the need for login
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    return pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

llm = load_model()

# Chat interface
user_prompt = st.text_area("💬 You:", height=100, placeholder="Ask Fusion AI anything...")

if st.button("Generate Response"):
    if user_prompt.strip() != "":
        with st.spinner("Fusion AI is thinking..."):
            response = llm(user_prompt)[0]["generated_text"]
            # Remove prompt from start of generated text if it repeats
            if response.startswith(user_prompt):
                response = response[len(user_prompt):]
            st.markdown(f"**🧠 Fusion AI:** {response.strip()}")
    else:
        st.warning("Please enter a question.")


2025-04-19 09:00:14.072 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 09:00:14.085 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 09:00:14.092 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 09:00:14.096 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 09:00:14.103 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 09:00:14.108 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 09:00:14.114 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 09:00:14.184 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar